In [1]:
import importlib
import frengression
# importlib.reload(frengression)
from data_causl.utils import *
from data_causl.data import *
from frengression import *

device = torch.device('cpu')
from CausalEGM import *

import numpy as np
import jax.numpy as jnp
import pickle
import os
from tqdm import tqdm
import src.exp_utils as exp_utils

from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import log_loss
from scipy.spatial.distance import pdist, squareform
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
s, x,z,y= generate_data_survivl(n=10,T=3)

In [3]:
s = torch.tensor(s, dtype=torch.float32)
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)
z = torch.tensor(z, dtype=torch.float32)

In [4]:
model = FrengressionSeq(x_dim=1, y_dim=1, z_dim=1, T=3, s_dim = 1, noise_dim=10, num_layer=3, hidden_dim=100, device=device, x_binary = False)

In [5]:
model.sample_xz(s=s, x=x, z=z, y = y)

(tensor([[ 0.0744,  0.0795, -0.0495],
         [ 0.1413,  0.0518, -0.1206],
         [ 0.1888,  0.1367, -0.2202],
         [ 0.1960,  0.0772, -0.1084],
         [ 0.1093,  0.1330, -0.1330],
         [ 0.0819,  0.0457, -0.2311],
         [ 0.1436,  0.0556, -0.0918],
         [ 0.0924,  0.0932, -0.0600],
         [ 0.2395,  0.0859, -0.0991],
         [ 0.1105,  0.1390, -0.1398]], grad_fn=<CatBackward0>),
 tensor([[ 0.1942,  0.0878,  0.0877],
         [ 0.0249, -0.0410,  0.0592],
         [-0.0982, -0.0835, -0.0602],
         [-0.0032, -0.0921, -0.0014],
         [ 0.0337, -0.1138, -0.0564],
         [ 0.0916,  0.1178, -0.0620],
         [-0.1375,  0.1587,  0.0753],
         [ 0.0505,  0.1494,  0.0018],
         [ 0.0208, -0.1692, -0.0390],
         [ 0.1315, -0.0034,  0.0368]], grad_fn=<CatBackward0>))

In [7]:
eta_try = model.sample_eta(x=x,z=z)

In [8]:
model.sample_y(x=x,z=z,eta=eta_try)

tensor([[-0.1164,  0.0345, -0.0610],
        [-0.1726, -0.0159, -0.0731],
        [-0.0598, -0.1156, -0.0662],
        [-0.0608,  0.0146, -0.1061],
        [-0.0940, -0.1299, -0.1508],
        [-0.0859, -0.1340,  0.0008],
        [ 0.0076, -0.0570, -0.1401],
        [ 0.0271, -0.0314, -0.0100],
        [-0.0974, -0.0466, -0.1284],
        [ 0.0605,  0.0649, -0.0890]], grad_fn=<CatBackward0>)

tensor([[ 0.5512, -1.8793,  0.8042],
        [ 1.7757, -1.2392,  0.0764],
        [ 2.0281,  1.1604,  1.0271],
        [ 1.3850,  0.0037,  1.7708],
        [ 0.3300, -0.0955, -0.4577],
        [ 0.1655,  1.7956,  1.3170],
        [ 0.9873, -1.3221, -0.3142],
        [ 1.4977, -0.2761, -0.4942],
        [ 0.6125, -0.7440, -0.0250],
        [ 0.6384, -1.0701, -0.8644]])

In [14]:
x[:,]

tensor([0.2428, 1.5665, 0.2144, 0.9146, 1.7378, 1.2524, 0.9293, 0.1718, 1.4671,
        0.8637])